In [21]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load the training data into feature matrix, class labels, and event ids:

In [22]:
from proj1_helpers import *
from data_analysis_logistic import *
from implementation import *
from cross_validation_reg_logistic import*

In [24]:
DATA_TRAIN_PATH = "data/train.csv" # download train data and supply path
y_tr, tX_tr, ids_tr = load_csv_data(DATA_TRAIN_PATH)

DATA_TEST_PATH = "data/test.csv" #download train data and supply path
y_fin, tX_fin, ids_fin = load_csv_data(DATA_TEST_PATH)

In [27]:
print(y_tr.shape)

(250000,)


# Implementation

## Separation of the data

In [25]:
jet_num = 0
y0_tr, tX0_tr,y0_tr_new, tX0_tr_new, ids0_tr, y0_te, tX0_te, ids0_te, y0_fin, tX0_fin, ids0_fin = data_analysis(jet_num, y_tr, tX_tr, ids_tr, y_fin, tX_fin, ids_fin)
jet_num = 1
y1_tr, tX1_tr,y1_tr_new, tX1_tr_new, ids1_tr,y1_te, tX1_te, ids1_te, y1_fin, tX1_fin, ids1_fin = data_analysis(jet_num, y_tr, tX_tr, ids_tr, y_fin, tX_fin, ids_fin)
jet_num = 2
y2_tr, tX2_tr,y2_tr_new, tX2_tr_new, ids2_tr,y2_te, tX2_te, ids2_te, y2_fin, tX2_fin, ids2_fin = data_analysis(jet_num, y_tr, tX_tr, ids_tr, y_fin, tX_fin, ids_fin)
jet_num = 3
y3_tr, tX3_tr,y3_tr_new, tX3_tr_new, ids3_tr,y3_te, tX3_te, ids3_te, y3_fin, tX3_fin, ids3_fin = data_analysis(jet_num, y_tr, tX_tr, ids_tr, y_fin, tX_fin, ids_fin)

In [26]:
print(y0_tr.shape)
print(y1_tr.shape)
print(y2_tr.shape)
print(y3_tr.shape)

(92898,)
(71338,)
(43964,)
(19115,)


## Initialization

In [19]:
init_w0=np.zeros(tX0_tr.shape[1])
init_w1=np.zeros(tX1_tr.shape[1])
init_w2=np.zeros(tX2_tr.shape[1])
init_w3=np.zeros(tX3_tr.shape[1])

init_w0_new=np.zeros(tX0_tr_new.shape[1])
init_w1_new=np.zeros(tX1_tr_new.shape[1])
init_w2_new=np.zeros(tX2_tr_new.shape[1])
init_w3_new=np.zeros(tX3_tr_new.shape[1])

best_lambdas = [0., 0., 0., 0.,]

max_iters=40
gamma=0.01

## No cross validation - no extended feature - no separation of data

Résultats:
- Categorical accuracy: 0.663
- F1-Score: 0.362
- Our grade: 0.642 - 0.657*

*Non testé

In [23]:
ratio=0.8
X_tr, X_te, Y_tr, Y_te, IDS_tr, IDS_te=split_data(tX_tr, y_tr, ids_tr, ratio, seed=1)
X_tr, m, std = standardize(X_tr)
X_te = standardize_te(X_te, m, std)
X_tr,IDS_tr,Y_tr=extract_outliers(X_tr,IDS_tr,Y_tr)
X_te,IDS_te,Y_te=extract_outliers(X_te,IDS_te,Y_te)

init_w=np.zeros(X_tr.shape[1])
best_lambda=test_lambda_cst(Y_tr,X_tr,Y_te,X_te,init_w, max_iters, gamma)

print(best_lambda)

6.723357536499335


In [24]:
w, loss = reg_logistic_regression(Y_tr, X_tr,init_w, best_lambda, max_iters, gamma)

y_pred = predict_labels(w, X_te)

res = np.where(Y_te[:,] == y_pred[:,], 1, 0)
grade = np.mean(res)

print(grade)

0.6566727928965258


In [33]:
init_w=np.zeros(tX_tr.shape[1])
w, loss = reg_logistic_regression(y_tr, tX_tr,init_w, best_lambda, max_iters, gamma)

y_pred = predict_labels(w, tX_fin)
y_pred=rescale_y(y_pred, ids_fin, 0,-1)
ids_pred = ids_fin
OUTPUT_PATH = 'results_reg_logistic_nocross_noext_nosep.csv' 
create_csv_submission(ids_pred, y_pred, OUTPUT_PATH)

## Cross validation of lambda - No extended feature

Résultats:
- Categorical accuracy: 0.687
- F1-Score: 0.591
- Our grade: 0.695 - 0.710*


A revoir: Les lambdas sont ils bien definies? Notamment pour lambda 0.

*pas encore soumis

In [14]:
best_lambdas[0]= cross_validation_lambda_reg_logistic(y0_tr, tX0_tr, max_iters, gamma)
best_lambdas[1]= cross_validation_lambda_reg_logistic(y1_tr, tX1_tr, max_iters, gamma)
best_lambdas[2]= cross_validation_lambda_reg_logistic(y2_tr, tX2_tr, max_iters, gamma)
best_lambdas[3]= cross_validation_lambda_reg_logistic(y3_tr, tX3_tr, max_iters, gamma)

print(best_lambdas)

[0.00379269019073225, 1e-05, 0.11288378916846883, 0.00379269019073225]


In [15]:
w0, loss0 = reg_logistic_regression(y0_tr_new, tX0_tr_new,init_w0_new, best_lambdas[0], max_iters, gamma)
w1, loss1 = reg_logistic_regression(y1_tr_new, tX1_tr_new,init_w1_new, best_lambdas[1], max_iters, gamma)
w2, loss2 = reg_logistic_regression(y2_tr_new, tX2_tr_new,init_w2_new, best_lambdas[2], max_iters, gamma)
w3, loss3 = reg_logistic_regression(y3_tr_new, tX3_tr_new,init_w3_new, best_lambdas[3], max_iters, gamma)

y0_pred = predict_labels(w0, tX0_te)
y1_pred = predict_labels(w1, tX1_te)
y2_pred = predict_labels(w2, tX2_te)
y3_pred = predict_labels(w3, tX3_te)

grades = [0., 0., 0., 0.]
res0 = np.where(y0_te[:,] == y0_pred[:,], 1, 0)
grades[0] = np.mean(res0)
res1 = np.where(y1_te[:,] == y1_pred[:,], 1, 0)
grades[1] = np.mean(res0)
res2 = np.where(y2_te[:,] == y2_pred[:,], 1, 0)
grades[2] = np.mean(res2)
res3 = np.where(y3_te[:,] == y3_pred[:,], 1, 0)
grades[3] = np.mean(res3)
grade = np.mean(grades)

print(grade)

0.7104516857520505


In [8]:
w0, loss0 = reg_logistic_regression(y0_tr, tX0_tr,init_w0, best_lambdas[0], max_iters, gamma)
w1, loss1 = reg_logistic_regression(y1_tr, tX1_tr,init_w1, best_lambdas[1], max_iters, gamma)
w2, loss2 = reg_logistic_regression(y2_tr, tX2_tr,init_w2, best_lambdas[2], max_iters, gamma)
w3, loss3 = reg_logistic_regression(y3_tr, tX3_tr,init_w3, best_lambdas[3], max_iters, gamma)


y0_pred = predict_labels(w0, tX0_fin)
y1_pred = predict_labels(w1, tX1_fin)
y2_pred = predict_labels(w2, tX2_fin)
y3_pred = predict_labels(w3, tX3_fin)

In [9]:
y_pred = np.concatenate([y0_pred, y1_pred, y2_pred, y3_pred])
ids_pred = np.concatenate([ids0_fin, ids1_fin, ids2_fin, ids3_fin])
y_pred=rescale_y(y_pred, ids_fin, 0,-1)
OUTPUT_PATH = 'results_reg_logistic_cross_noext_sep.csv' 
create_csv_submission(ids_pred, y_pred, OUTPUT_PATH)

## Cross validation - Extended feature

Résultat:
- Categorical accuracy:0.744
- F1-Score: 0.463
- Our grade: 0.760 - 0.731*

In [16]:
lambda_=6.72

deg0=cross_validation_deg_reg_logistic(y0_tr, tX0_tr, max_iters, gamma,lambda_)
deg1=cross_validation_deg_reg_logistic(y1_tr, tX1_tr, max_iters, gamma,lambda_)
deg2=cross_validation_deg_reg_logistic(y2_tr, tX2_tr, max_iters, gamma,lambda_)
deg3=cross_validation_deg_reg_logistic(y3_tr, tX3_tr, max_iters, gamma,lambda_)

print(deg0)
print(deg1)
print(deg2)
print(deg3)

[4 1 1 1 1 1 1 3 1 1 1 1 1 1 1 1 1 1]
[4 1 4 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[4 1 1 1 3 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[4 3 4 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 3]


In [17]:
tX0_tr,tX0_tr_new,tX0_te,tX0_fin=build_poly_data(tX0_tr,tX0_tr_new,tX0_te,tX0_fin,deg0)
tX1_tr,tX1_tr_new,tX1_te,tX1_fin=build_poly_data(tX1_tr,tX1_tr_new,tX1_te,tX1_fin,deg1)
tX2_tr,tX2_tr_new,tX2_te,tX2_fin=build_poly_data(tX2_tr,tX2_tr_new,tX2_te,tX2_fin,deg2)
tX3_tr,tX3_tr_new,tX3_te,tX3_fin=build_poly_data(tX3_tr,tX3_tr_new,tX3_te,tX3_fin,deg3)

In [18]:
best_lambdas[0]= cross_validation_lambda_reg_logistic(y0_tr, tX0_tr, max_iters, gamma)
best_lambdas[1]= cross_validation_lambda_reg_logistic(y1_tr, tX1_tr, max_iters, gamma)
best_lambdas[2]= cross_validation_lambda_reg_logistic(y2_tr, tX2_tr, max_iters, gamma)
best_lambdas[3]= cross_validation_lambda_reg_logistic(y3_tr, tX3_tr, max_iters, gamma)

print(best_lambdas)

[100.0, 1e-05, 100.0, 18.32980710832434]


In [20]:
w0, loss0 = reg_logistic_regression(y0_tr_new, tX0_tr_new,init_w0_new, best_lambdas[0], max_iters, gamma)
w1, loss1 = reg_logistic_regression(y1_tr_new, tX1_tr_new,init_w1_new, best_lambdas[1], max_iters, gamma)
w2, loss2 = reg_logistic_regression(y2_tr_new, tX2_tr_new,init_w2_new, best_lambdas[2], max_iters, gamma)
w3, loss3 = reg_logistic_regression(y3_tr_new, tX3_tr_new,init_w3_new, best_lambdas[3], max_iters, gamma)

y0_pred = predict_labels(w0, tX0_te)
y1_pred = predict_labels(w1, tX1_te)
y2_pred = predict_labels(w2, tX2_te)
y3_pred = predict_labels(w3, tX3_te)


grades = [0., 0., 0., 0.]
res0 = np.where(y0_te[:,] == y0_pred[:,], 1, 0)
grades[0] = np.mean(res0)
res1 = np.where(y1_te[:,] == y1_pred[:,], 1, 0)
grades[1] = np.mean(res0)
res2 = np.where(y2_te[:,] == y2_pred[:,], 1, 0)
grades[2] = np.mean(res2)
res3 = np.where(y3_te[:,] == y3_pred[:,], 1, 0)
grades[3] = np.mean(res3)
grade = np.mean(grades)

print(grade)

0.7315074500544856


In [31]:
w0, loss0 = reg_logistic_regression(y0_tr, tX0_tr,init_w0, best_lambdas[0], max_iters, gamma)
w1, loss1 = reg_logistic_regression(y1_tr, tX1_tr,init_w1, best_lambdas[1], max_iters, gamma)
w2, loss2 = reg_logistic_regression(y2_tr, tX2_tr,init_w2, best_lambdas[2], max_iters, gamma)
w3, loss3 = reg_logistic_regression(y3_tr, tX3_tr,init_w3, best_lambdas[3], max_iters, gamma)


y0_pred = predict_labels(w0, tX0_fin)
y1_pred = predict_labels(w1, tX1_fin)
y2_pred = predict_labels(w2, tX2_fin)
y3_pred = predict_labels(w3, tX3_fin)

In [32]:
y_pred = np.concatenate([y0_pred, y1_pred, y2_pred, y3_pred])
y_pred=rescale_y(y_pred, ids_fin, 0,-1)
ids_pred = np.concatenate([ids0_fin, ids1_fin, ids2_fin, ids3_fin])
OUTPUT_PATH = 'results_reg_logistic_cross_ext_sep.csv' 
create_csv_submission(ids_pred, y_pred, OUTPUT_PATH)